In [1]:
from tensorflow import keras
import numpy as np
import tensorflow as tf
import json
from CocktailEmbeddingMaker import CocktailEmbeddingMaker
from CocktailEmbeddingMaker import Eval
# 데이터 로드
with open('./train_data.json', 'r') as f:
    json_data = json.load(f)
with open('../flavor.json', 'r') as f:
    flavor_data = json.load(f)
with open('../category.json', 'r') as f:
    category_data = json.load(f)
with open('user_list_v1_5.json', 'r') as f:
    test_user_list = json.load(f)

attributes = ['ABV', 'boozy', 'sweet', 'sour', 'bitter', 'umami', 'salty', 'astringent', 'Perceived_temperature', 'spicy', 'herbal', 'floral', 'fruity', 'nutty', 'creamy', 'smoky']
#가상의 유저 만들기
test_user_list = []
user_num = 5
for i in range(user_num):
    user = {}
    # user['user_id'] = i
    preference = {}
    user['ABV'] = np.random.randint(0,60)
    for attribute in attributes[1:]:
        user[attribute] = np.random.randint(0,100)
    test_user_list.append(user)

#음료의 총 량
total_amount = 200
#모델 로드
#D:\CODE\feelflask_master\backend\data_works\bidirectional_lstm.h5
# load_model = keras.models.load_model('bidirectional_lstm.h5')
load_model = keras.models.load_model('best_model_earthy.h5')
#Eval 객체 생성 -> inference용 전용 class가 필요하면 이야기할것
Eval_obj = Eval(json_data,flavor_data,category_data,load_model)
list_result_2 = []
for i in range(user_num):
    # print(f"[OUTPUT]user_recipe's profile : {json.dumps(user_recipe_profile,indent=4)}")
    user_seed = Eval_obj.select_user_seed(test_user_list[i])
    generated_recipes = Eval_obj.generate_recipe(load_model,user_seed,test_user_list[i])
    result_recipe ={}

    for recipe, ingredients in zip(generated_recipes[0], generated_recipes[1]):
        result_recipe[recipe]=ingredients * total_amount
    user_recipe_profile = Eval_obj.get_taste_log(generated_recipes)

    print(" please ignore user_id!")
    print(f"[INPUT] user preference: {json.dumps(test_user_list[i],indent=4)}")
    print(f"[OUTPUT]recipe!!!!: {result_recipe} ")
    list_result_2.append(result_recipe)

    limited_ingredient_list= ['peach schnapps','baileys irish cream','kahlua','triple sec','malibu rum','tequila',
                              'whisky','jack daniels','cherry grenadine','malibu rum','midori melon liqueur','salt','sugar','vodka','rum']
    Eval_obj.set_limited_ingredient(limited_ingredient_list)
    limited_generated_recipe, limited_quantities = Eval_obj.generate_recipe_limited(load_model, user_seed, test_user_list[i])
    print(f"제한된 재료로 생성한 레시피: {limited_generated_recipe}")



user_seed : rumple minze
seed_ingredient : rumple minze
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 312ms/step
next_ingredient : kahlua, total_prob : 0 , normalized_prob[next_ingredient_id] : 0.16954053446611403
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
next_ingredient : baileys irish cream, total_prob : 0.16954053446611403 , normalized_prob[next_ingredient_id] : 0.23409494090111324
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
next_ingredient : grand marnier, total_prob : 0.40363547536722727 , normalized_prob[next_ingredient_id] : 0.24270087197710294
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
next_ingredient : gin, total_prob : 0.6463363473443302 , normalized_prob[next_ingredient_id] : 0.28283173428874026
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
next_ingredient : vodka, total_prob : 0.9291680816330705 , normalized_prob[next_ingredient_id] : 0.3538199118507918
 please ignore user_id!
[INPUT] user preference: {
    "ABV": 55,
    "boozy": 39,
    "sweet": 35,
    "sour": 73,
    "bitter": 63,
    "umami": 46,
    "salty":

In [ ]:
category_data

# 카테고리별로 재료를 저장할 딕셔너리 초기화
categorized_ingredients = {}

# 각 재료를 반복하면서 카테고리별로 분류
for ingredient, categories in category_data.items():
    for category in categories:
        if category not in categorized_ingredients:
            categorized_ingredients[category] = []
        categorized_ingredients[category].append(ingredient)

# 결과 출력
for category, ingredients in categorized_ingredients.items():
    print(f"Category: {category}")
    print("Ingredients:")
    for ingredient in ingredients:
        print(f"{ingredient}")
    print()

In [ ]:
import json
with open('../category.json', 'r') as f:
    category_data = json.load(f)
# JSON 문자열을 파싱하여 딕셔너리로 변환
# 카테고리별로 재료를 저장할 딕셔너리 초기화
categorized_ingredients = {}

# 각 재료를 반복하면서 카테고리별로 분류
for ingredient, categories in category_data.items():
    for category in categories:
        if category not in categorized_ingredients:
            categorized_ingredients[category] = []
        categorized_ingredients[category].append(ingredient)

# 결과 출력
for category, ingredients in categorized_ingredients.items():
    print(f"Category: {category}")
    print("Ingredients:")
    for ingredient in ingredients:
        print(f"- {ingredient}")
    print()

In [ ]:
category_data